In [26]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F9DFCF8790>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F9DFCE66B0>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

langchain_huggingace and setence_transformers is in requirements.txt

using the 'all-MiniLM-L6-2' becomes easy when we hae sentenceTransformer installed

and also MiniLM is a sentence transformer only


In [28]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

Documents

1. page_content
2. metadata

In [29]:
from langchain_core.documents import Document

docs = [
    Document(
        page_content="Artificial Intelligence is the simulation of human intelligence in machines.",
        metadata={"source": "Wikipedia", "topic": "AI", "page": 1}
    ),
    Document(
        page_content="Machine learning is a subset of AI focused on algorithms that improve over time.",
        metadata={"source": "Encyclopedia Britannica", "topic": "ML", "page": 2}
    ),
    Document(
        page_content="Deep learning uses neural networks with many layers to model complex patterns in data.",
        metadata={"source": "Research Paper", "topic": "Deep Learning", "page": 3}
    ),
    Document(
        page_content="Natural language processing allows machines to understand and generate human language.",
        metadata={"source": "Tech Blog", "topic": "NLP", "page": 4}
    ),
    Document(
        page_content="Computer vision enables systems to interpret and process visual information from the world.",
        metadata={"source": "AI Magazine", "topic": "Computer Vision", "page": 5}
    ),
    Document(
        page_content="Reinforcement learning is a paradigm where agents learn by receiving rewards or penalties.",
        metadata={"source": "Online Course", "topic": "RL", "page": 6}
    )
]


docs

[Document(metadata={'source': 'Wikipedia', 'topic': 'AI', 'page': 1}, page_content='Artificial Intelligence is the simulation of human intelligence in machines.'),
 Document(metadata={'source': 'Encyclopedia Britannica', 'topic': 'ML', 'page': 2}, page_content='Machine learning is a subset of AI focused on algorithms that improve over time.'),
 Document(metadata={'source': 'Research Paper', 'topic': 'Deep Learning', 'page': 3}, page_content='Deep learning uses neural networks with many layers to model complex patterns in data.'),
 Document(metadata={'source': 'Tech Blog', 'topic': 'NLP', 'page': 4}, page_content='Natural language processing allows machines to understand and generate human language.'),
 Document(metadata={'source': 'AI Magazine', 'topic': 'Computer Vision', 'page': 5}, page_content='Computer vision enables systems to interpret and process visual information from the world.'),
 Document(metadata={'source': 'Online Course', 'topic': 'RL', 'page': 6}, page_content='Reinfor

In [30]:
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(docs, embeddings)
vectorstore

d:\GenAI\genv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [31]:
vectorstore.similarity_search('AI')

[Document(id='b7043fb8-a072-4fac-8c7f-08d9c53d0e8a', metadata={'source': 'Wikipedia', 'topic': 'AI', 'page': 1}, page_content='Artificial Intelligence is the simulation of human intelligence in machines.'),
 Document(id='22a89554-0107-4eb3-9b61-0773581e50f5', metadata={'page': 1, 'source': 'Wikipedia', 'topic': 'AI'}, page_content='Artificial Intelligence is the simulation of human intelligence in machines.'),
 Document(id='64774d94-4557-4627-ac98-a79ef9689bc9', metadata={'source': 'Encyclopedia Britannica', 'page': 2, 'topic': 'ML'}, page_content='Machine learning is a subset of AI focused on algorithms that improve over time.'),
 Document(id='75030ba5-0d3f-4698-a81e-7b4cdf2ea024', metadata={'topic': 'ML', 'page': 2, 'source': 'Encyclopedia Britannica'}, page_content='Machine learning is a subset of AI focused on algorithms that improve over time.')]

Async qury

In [32]:
await vectorstore.asimilarity_search('AI')

[Document(id='b7043fb8-a072-4fac-8c7f-08d9c53d0e8a', metadata={'topic': 'AI', 'page': 1, 'source': 'Wikipedia'}, page_content='Artificial Intelligence is the simulation of human intelligence in machines.'),
 Document(id='22a89554-0107-4eb3-9b61-0773581e50f5', metadata={'source': 'Wikipedia', 'topic': 'AI', 'page': 1}, page_content='Artificial Intelligence is the simulation of human intelligence in machines.'),
 Document(id='64774d94-4557-4627-ac98-a79ef9689bc9', metadata={'page': 2, 'topic': 'ML', 'source': 'Encyclopedia Britannica'}, page_content='Machine learning is a subset of AI focused on algorithms that improve over time.'),
 Document(id='75030ba5-0d3f-4698-a81e-7b4cdf2ea024', metadata={'topic': 'ML', 'page': 2, 'source': 'Encyclopedia Britannica'}, page_content='Machine learning is a subset of AI focused on algorithms that improve over time.')]

# Retriever

vectorstores are not part of runnable  and cnt direclty used in LCEL, there is a workaround, ues as_retriever , its a runnable

In [33]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

In [ ]:
# METHOD 1
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)# top 1 sim search bot all words/query in batch
retriever.batch(["AI", "nlp"])

d:\GenAI\genv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[[Document(id='b7043fb8-a072-4fac-8c7f-08d9c53d0e8a', metadata={'source': 'Wikipedia', 'topic': 'AI', 'page': 1}, page_content='Artificial Intelligence is the simulation of human intelligence in machines.')],
 [Document(id='18ce09bb-1c10-4da7-b837-655fd4859fb1', metadata={'topic': 'NLP', 'page': 4, 'source': 'Tech Blog'}, page_content='Natural language processing allows machines to understand and generate human language.')]]

In [ ]:
# METHOD 2 , more used 
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}# top 1 for all word/query
)

retriever.batch(["AI","ML"])





# demo
#retriever = db.as_retriever()  
#doc2 = retriever.invoke(query)
#doc2

d:\GenAI\genv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[[Document(id='b7043fb8-a072-4fac-8c7f-08d9c53d0e8a', metadata={'source': 'Wikipedia', 'page': 1, 'topic': 'AI'}, page_content='Artificial Intelligence is the simulation of human intelligence in machines.')],
 [Document(id='64774d94-4557-4627-ac98-a79ef9689bc9', metadata={'source': 'Encyclopedia Britannica', 'page': 2, 'topic': 'ML'}, page_content='Machine learning is a subset of AI focused on algorithms that improve over time.')]]

---------------------

# Basic Rag

In [37]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
answer this question using the provided context only.
{question}
context:{context}
"""


prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question":RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about AI")
print(response.content)

d:\GenAI\genv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


According to the provided context, Artificial Intelligence (AI) is the simulation of human intelligence in machines.
